# Import
libraries + api keys

In [135]:
import geopandas as gpd
from geopy.distance import geodesic as gd
import requests
import folium
import openrouteservice as ors
import API_KEYS

# Download data from bike station life feed

In [136]:
URL='http://bikeshare.metro.net/stations/json/'
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"
headers = {"user-agent": USER_AGENT} # adding the user agent
resp = requests.get(URL, headers=headers)
f = open("map_data.json", "w")
f.write(resp.text)
f.close()

# Parsing data from geojson to dict

In [137]:
file = open('map_data.json')
read = gpd.read_file(file)
read = read.reset_index()
#dictionary with neccesary informations  - street, location, num of available bikes, num of available bikes, distance from user (-1 for not count)
station_info = {}
for index, row in read.iterrows():
    coordinates = (row['latitude'], row['longitude'])
    station_info[row['addressStreet']] = {'coordinates': coordinates, 'bikes': row['bikesAvailable'], 'docks': row['docksAvailable'], 'distance':-1}
station_info

{'700 Flower St': {'coordinates': (34.0485, -118.25854),
  'bikes': 12,
  'docks': 19,
  'distance': -1},
 '729 S Olive Street': {'coordinates': (34.04554, -118.25667),
  'bikes': 13,
  'docks': 18,
  'distance': -1},
 '557 S 5th Street': {'coordinates': (34.05048, -118.25459),
  'bikes': 1,
  'docks': 22,
  'distance': -1},
 '865 S Figueroa Street': {'coordinates': (34.04661, -118.26273),
  'bikes': 6,
  'docks': 8,
  'distance': -1},
 '401 East 11th Street': {'coordinates': (34.03705, -118.25487),
  'bikes': 9,
  'docks': 6,
  'distance': -1},
 '1259-1299 S Figueroa Street': {'coordinates': (34.04113, -118.26798),
  'bikes': 8,
  'docks': 11,
  'distance': -1},
 '834 N Alameda St': {'coordinates': (34.05661, -118.23721),
  'bikes': 11,
  'docks': 22,
  'distance': -1},
 '201 N Los Angeles Street': {'coordinates': (34.0529, -118.24156),
  'bikes': 4,
  'docks': 26,
  'distance': -1},
 '919 S Grand Ave.': {'coordinates': (34.04373, -118.26014),
  'bikes': 2,
  'docks': 17,
  'distance'

{'700 Flower St': {'coordinates': (34.0485, -118.25854),
  'bikes': 12,
  'docks': 19,
  'distance': -1},
 '729 S Olive Street': {'coordinates': (34.04554, -118.25667),
  'bikes': 13,
  'docks': 18,
  'distance': -1},
 '557 S 5th Street': {'coordinates': (34.05048, -118.25459),
  'bikes': 1,
  'docks': 22,
  'distance': -1},
 '865 S Figueroa Street': {'coordinates': (34.04661, -118.26273),
  'bikes': 6,
  'docks': 8,
  'distance': -1},
 '401 East 11th Street': {'coordinates': (34.03705, -118.25487),
  'bikes': 9,
  'docks': 6,
  'distance': -1},
 '1259-1299 S Figueroa Street': {'coordinates': (34.04113, -118.26798),
  'bikes': 8,
  'docks': 11,
  'distance': -1},
 '834 N Alameda St': {'coordinates': (34.05661, -118.23721),
  'bikes': 11,
  'docks': 22,
  'distance': -1},
 '201 N Los Angeles Street': {'coordinates': (34.0529, -118.24156),
  'bikes': 4,
  'docks': 26,
  'distance': -1},
 '919 S Grand Ave.': {'coordinates': (34.04373, -118.26014),
  'bikes': 2,
  'docks': 17,
  'distance'

# Reseet distance info

In [138]:
# reset the distance info - the same structure might be used multiple times
def reset_station_distance(station_info):
    for item in station_info.items():
        item[1]['distance']=-1

# Find nearest station with available bikes

In [139]:
def find_nearest_stations_with_bikes(lat_user, lon_user, k_stations, station_info):
    points=[]
    map = folium.Map(location=[34.04850,-118.25854 ], zoom_start=15)
    icon = folium.Icon(color="red")
    folium.Marker(location=[34.04862,-118.25874 ], icon=icon).add_to(map)
    reset_station_distance(station_info) # be sure that structure is empty and ready to use
    user_position =(lat_user,lon_user)
    for item in station_info.items():
        if item[1]['bikes'] >0: # count distance for stations with available bikes
            item[1]['distance']=gd(user_position,item[1]['coordinates']).kilometers
    station_info = sorted(station_info.items(), key=lambda item: item[1]['distance'], reverse=False)
    # print the nearest stations -  skip the stations without distance
    station_counter=0
    for item in station_info:
        if item[1]['distance'] != -1:
            points.append(item[1]['coordinates'])
            print(item)
            station_counter+=1
        if station_counter==k_stations: # check how many stations was already printed
            break

    for point in range(0, len(points)):
        folium.Marker(points[point]).add_to(map)

     #folium.Marker([item[1]['coordinates'][0], item[1]['coordinates'][1]], icon=icon).add_to(map)
    map.save('index.html')

    return station_info

find_nearest_stations_with_bikes(34.04850,-118.25854,10, station_info)



('700 Flower St', {'coordinates': (34.0485, -118.25854), 'bikes': 12, 'docks': 19, 'distance': 0.0})
('735 S Figueroa Street', {'coordinates': (34.0484, -118.26095), 'bikes': 7, 'docks': 16, 'distance': 0.2227971446605152})
('786 S Grand Ave', {'coordinates': (34.04613, -118.25759), 'bikes': 4, 'docks': 14, 'distance': 0.2771360494335669})
('557 S Hope Street', {'coordinates': (34.04989, -118.25588), 'bikes': 8, 'docks': 23, 'distance': 0.28998747967863})
('729 S Olive Street', {'coordinates': (34.04554, -118.25667), 'bikes': 13, 'docks': 18, 'distance': 0.3709654625317443})
('557 S 5th Street', {'coordinates': (34.05048, -118.25459), 'bikes': 1, 'docks': 22, 'distance': 0.42573293044972493})
('865 S Figueroa Street', {'coordinates': (34.04661, -118.26273), 'bikes': 6, 'docks': 8, 'distance': 0.4400276759578756})
('532 W Olive Street', {'coordinates': (34.04804, -118.25374), 'bikes': 6, 'docks': 16, 'distance': 0.44612352883657697})
('960 S Hope Street', {'coordinates': (34.04417, -118

[('129 W 12th Street',
  {'coordinates': (34.03861, -118.26086),
   'bikes': 0,
   'docks': 19,
   'distance': -1}),
 ('700 S Hill St',
  {'coordinates': (34.04585, -118.2544),
   'bikes': 0,
   'docks': 19,
   'distance': -1}),
 ('700 Flower St',
  {'coordinates': (34.0485, -118.25854),
   'bikes': 12,
   'docks': 19,
   'distance': 0.0}),
 ('735 S Figueroa Street',
  {'coordinates': (34.0484, -118.26095),
   'bikes': 7,
   'docks': 16,
   'distance': 0.2227971446605152}),
 ('786 S Grand Ave',
  {'coordinates': (34.04613, -118.25759),
   'bikes': 4,
   'docks': 14,
   'distance': 0.2771360494335669}),
 ('557 S Hope Street',
  {'coordinates': (34.04989, -118.25588),
   'bikes': 8,
   'docks': 23,
   'distance': 0.28998747967863}),
 ('729 S Olive Street',
  {'coordinates': (34.04554, -118.25667),
   'bikes': 13,
   'docks': 18,
   'distance': 0.3709654625317443}),
 ('557 S 5th Street',
  {'coordinates': (34.05048, -118.25459),
   'bikes': 1,
   'docks': 22,
   'distance': 0.425732930449

('700 Flower St', {'coordinates': (34.0485, -118.25854), 'bikes': 12, 'docks': 19, 'distance': 0.0})
('735 S Figueroa Street', {'coordinates': (34.0484, -118.26095), 'bikes': 7, 'docks': 16, 'distance': 0.2227971446605152})
('786 S Grand Ave', {'coordinates': (34.04613, -118.25759), 'bikes': 4, 'docks': 14, 'distance': 0.2771360494335669})
('557 S Hope Street', {'coordinates': (34.04989, -118.25588), 'bikes': 8, 'docks': 23, 'distance': 0.28998747967863})
('729 S Olive Street', {'coordinates': (34.04554, -118.25667), 'bikes': 13, 'docks': 18, 'distance': 0.3709654625317443})
('557 S 5th Street', {'coordinates': (34.05048, -118.25459), 'bikes': 1, 'docks': 22, 'distance': 0.42573293044972493})
('865 S Figueroa Street', {'coordinates': (34.04661, -118.26273), 'bikes': 6, 'docks': 8, 'distance': 0.4400276759578756})
('532 W Olive Street', {'coordinates': (34.04804, -118.25374), 'bikes': 6, 'docks': 16, 'distance': 0.44612352883657697})
('960 S Hope Street', {'coordinates': (34.04417, -118

[('129 W 12th Street',
  {'coordinates': (34.03861, -118.26086),
   'bikes': 0,
   'docks': 19,
   'distance': -1}),
 ('700 S Hill St',
  {'coordinates': (34.04585, -118.2544),
   'bikes': 0,
   'docks': 19,
   'distance': -1}),
 ('700 Flower St',
  {'coordinates': (34.0485, -118.25854),
   'bikes': 12,
   'docks': 19,
   'distance': 0.0}),
 ('735 S Figueroa Street',
  {'coordinates': (34.0484, -118.26095),
   'bikes': 7,
   'docks': 16,
   'distance': 0.2227971446605152}),
 ('786 S Grand Ave',
  {'coordinates': (34.04613, -118.25759),
   'bikes': 4,
   'docks': 14,
   'distance': 0.2771360494335669}),
 ('557 S Hope Street',
  {'coordinates': (34.04989, -118.25588),
   'bikes': 8,
   'docks': 23,
   'distance': 0.28998747967863}),
 ('729 S Olive Street',
  {'coordinates': (34.04554, -118.25667),
   'bikes': 13,
   'docks': 18,
   'distance': 0.3709654625317443}),
 ('557 S 5th Street',
  {'coordinates': (34.05048, -118.25459),
   'bikes': 1,
   'docks': 22,
   'distance': 0.425732930449

# finde nearest station with available docks

In [140]:
def find_nearest_stations_with_docks(lat_user, lon_user, k_stations, station_info):
    reset_station_distance(station_info)
    my_position =(lat_user,lon_user)
    for item in station_info.items():
        #print(item[1]['coordinates'])
        if item[1]['docks'] >0:
            item[1]['distance']=gd(my_position,item[1]['coordinates']).kilometers
    station_info = sorted(station_info.items(), key=lambda item: item[1]['distance'], reverse=False)
    station_counter=0
    for item in station_info:
        if item[1]['distance'] != -1:
            print(item)
            station_counter+=1
        if station_counter==k_stations:
            break

find_nearest_stations_with_docks(34.04850,-118.25854,2, station_info)

('700 Flower St', {'coordinates': (34.0485, -118.25854), 'bikes': 12, 'docks': 19, 'distance': 0.0})
('735 S Figueroa Street', {'coordinates': (34.0484, -118.26095), 'bikes': 7, 'docks': 16, 'distance': 0.2227971446605152})
('700 Flower St', {'coordinates': (34.0485, -118.25854), 'bikes': 12, 'docks': 19, 'distance': 0.0})
('735 S Figueroa Street', {'coordinates': (34.0484, -118.26095), 'bikes': 7, 'docks': 16, 'distance': 0.2227971446605152})


# Display route on the map

In [141]:
ors_key = API_KEYS.OPEN_ROUTE
client = ors.Client(key=ors_key)
# coordinates, long,lat
coordinates=((-118.25854, 34.04850), (-118.25667,34.02851))
#route
route = client.directions(coordinates=coordinates, profile ='foot-walking', format = 'geojson')
# map
map_direction = folium.Map(location=[34.04850,-118.25854 ], zoom_start=13)
# add route to the map
folium.GeoJson(route, name='route').add_to(map_direction)
# add layer control
folium.LayerControl().add_to(map_direction)
#display map
map_direction

# Otazky
Jaka ma byt finalni podoba - jenom skripty, nebo UI aplikace...
jaky je parametr pro posledi ukol, podle ceeho mam trasu vybrat, jak se ma fnce zavolat
kde mam vzit polohu uzivatele, kde mam vzit koncovy bod
nejake dalsi pozadavky na algoritmus
https://www.youtube.com/watch?v=mgC-NMpegYg